In [1]:
import pandas as pd
import allel
import numpy as np
import scipy.special
import polars as pr

In [2]:
test_vcf = allel.read_vcf('/mnt/hpccs01/scratch/microbiome/n10853499/02-lorikeet_testing/04-instrain_zymo/test/Bacillus_subtilis_complete_genome/Bacillus_subtilis_complete_genome.vcf', 
                          fields=['variants/CHROM', 'variants/POS', 'variants/DP', 'calldata/GT', 'calldata/AD', 'calldata/DP'])
# test_vcf = allel.read_vcf('sample.vcf')
# test_vcf = allel.read_vcf('/mnt/hpccs01/scratch/microbiome/n10853499/02-lorikeet_testing/02-mflorens_variant_test/03-lorikeet_out/GCA_003139855.1_20110800_S2M_genomic/GCA_003139855.1_20110800_S2M_genomic.vcf.gz')
# test_vcf = allel.read_vcf('/mnt/hpccs01/scratch/microbiome/n10853499/02-lorikeet_testing/02-mflorens_variant_test/test/GCA_003139855.1_20110800_S2M_genomic/GCA_003139855.1_20110800_S2M_genomic.vcf')
test_vcf['calldata/DP'].shape

(387, 3)

In [64]:
# hudson fst for haploid individual using allele depth as population size
output_prefix = "/mnt/hpccs01/scratch/microbiome/n10853499/02-lorikeet_testing/02-mflorens_variant_test/03.3-lorikeet_fst/GCA_003139855.1_20110800_S2M_genomic/"
genome_name = "GCA_003139855.1_20110800_S2M_genomic"
vcf = allel.read_vcf(f'{output_prefix}/{genome_name}.vcf.gz', 
                          fields=['variants/CHROM', 'variants/POS', 'variants/DP', 'variants/QF', 'calldata/GT', 'calldata/AD', 'calldata/DP'])

# print(vcf['calldata/AD'][vcf['variants/QF'] == 'true'])

depth_per_sample = int("3")
col_dict = ['sample_1', 'sample_2']
n_variants = 0
for variant_i in range(vcf['calldata/AD'].shape[0]):
#     if vcf['variants/QF'][variant_i] == 'false':
#             continue # do not include unqualified variants
    col_dict.append(f'variant_{variant_i}')
    n_variants += 1

# fst_df = np.zeros(shape=(int(scipy.special.binom(vcf['calldata/DP'].shape[1], 2)), n_variants + 2))
mean_fst_df = np.zeros(shape=(vcf['calldata/DP'].shape[1], vcf['calldata/DP'].shape[1]))
row = 0

allele_counts = vcf['calldata/AD'][vcf['variants/QF'] == 'true']
population_size = vcf['calldata/DP'][vcf['variants/QF'] == 'true']

def hudson_fst_op(df, x, y):  
    h1 = allel.HaplotypeArray([df[:, x][df[:, x] != -1]])
# #         print(h1)
    h2 = allel.HaplotypeArray([df[:, y][df[:, y] != -1]])
    num, den = allel.hudson_fst(h1, h2)
    fst = (num / den)

    np.nan_to_num(fst, copy=False, nan=0.0, posinf=None, neginf=None)
    fst[fst < 0] = 0

    mean_fst = max(fst_df[row, 2:].mean(), 0)
    
    return (fst, mean_fst)


for sample1 in range(vcf['calldata/DP'].shape[1]):
    for sample2 in range(sample1+1):
        if sample1 == sample2:
            continue

        # sample1_depths = np.rot90(np.repeat([population_size[:, sample1]], 2, axis=0))
        sample1_depths = population_size[:, sample1]
        sample1_allele_counts = allele_counts[:, sample1, (allele_counts!=-1)[:, sample1].all(axis=0)]
        # sample2_depths = np.rot90(np.repeat([population_size[:, sample2]], 2, axis=0))
        sample2_depths = population_size[:, sample2]
        sample2_allele_counts = allele_counts[:, sample2, (allele_counts!=-1)[:, sample2].all(axis=0)]
        
        variants_to_include_1 = (sample1_depths > depth_per_sample) #+ ((sample1_allele_counts / sample1_depths) > 0.01)
        variants_to_include_2 = (sample2_depths > depth_per_sample) #+ ((sample2_allele_counts / sample2_depths) > 0.01)
        variants_to_include = np.array([variants_to_include_1, variants_to_include_2]).all(axis=0)
        
        # print(sample1_allele_counts[variants_to_include].shape)
        # print(sample2_allele_counts[variants_to_include].shape)
        h1 = allel.HaplotypeArray(sample1_allele_counts[variants_to_include])
        h2 = allel.HaplotypeArray(sample2_allele_counts[variants_to_include])
        
        num, den = allel.hudson_fst(h1, h2)
        fst = (num / den)
        # print(fst)
        np.nan_to_num(fst, copy=False, nan=0.0, posinf=None, neginf=None)
        fst[fst < 0] = 0

        mean_fst_df[sample1, sample2] = fst.mean()
        mean_fst_df[sample2, sample1] = fst.mean()
        row += 1



samples = [str(i + 1) for i in range(vcf['calldata/DP'].shape[1])]
np.nan_to_num(mean_fst_df, copy=False, nan=0, posinf=0, neginf=0)
mean_fst_df = np.insert(mean_fst_df, 0, np.array([i + 1 for i in range(vcf['calldata/DP'].shape[1])]), axis=1)
samples.insert(0, "SampleID")
mean_fst_df = pr.DataFrame(mean_fst_df, columns=samples)
print(mean_fst_df)


/tmp/ipykernel_17939/3611294761.py:62: RuntimeWarning: invalid value encountered in true_divide
  fst = (num / den)
/tmp/ipykernel_17939/3611294761.py:67: RuntimeWarning: Mean of empty slice.
  mean_fst_df[sample1, sample2] = fst.mean()
/home/n10853499/.conda/envs/r-4.1/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_17939/3611294761.py:68: RuntimeWarning: Mean of empty slice.
  mean_fst_df[sample2, sample1] = fst.mean()
/tmp/ipykernel_17939/3611294761.py:62: RuntimeWarning: divide by zero encountered in true_divide
  fst = (num / den)


shape: (208, 209)
┌───────┬────────────┬───────────┬───────────┬─────┬───────────┬───────────┬───────────┬───────────┐
│ Sampl ┆ 1          ┆ 2         ┆ 3         ┆ ... ┆ 205       ┆ 206       ┆ 207       ┆ 208       │
│ eID   ┆ ---        ┆ ---       ┆ ---       ┆     ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ ---   ┆ f64        ┆ f64       ┆ f64       ┆     ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
│ f64   ┆            ┆           ┆           ┆     ┆           ┆           ┆           ┆           │
╞═══════╪════════════╪═══════════╪═══════════╪═════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 1.0   ┆ 0.0        ┆ 0.011684  ┆ 0.020747  ┆ ... ┆ 0.405678  ┆ 0.244444  ┆ 0.548979  ┆ 0.355556  │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┤
│ 2.0   ┆ 0.011684   ┆ 0.0       ┆ 0.006323  ┆ ... ┆ 0.483333  ┆ 0.25      ┆ 0.693878  ┆ 0.0       │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌

In [ ]:
mean_fst_df.to_csv()

In [36]:
# mean_fst_df[mean_fst_df > 0.25]

NotImplementedError: 

In [26]:
from glob import glob
glob("/mnt/hpccs01/scratch/microbiome/n10853499/02-lorikeet_testing/02-mflorens_variant_test/01-sra_samples/SRR12951261_1*")
print(allele_counts[:, sample1].shape)
print((allele_counts!=-1)[:, sample1].all(axis=0))
print(sample1_allele_counts.shape)

(25050, 4)
[ True  True False False]
(51014,)


In [57]:
# pr_df.write_csv(file=f"{output_prefix}/{genome_name}_fst_values.tsv", sep='\t')
# print(np.rot90(np.repeat(np.array([vcf['calldata/DP'][vcf['variants/QF'] == 'true', sample1]]), 4, axis=0)).shape)
# print((allele_counts[:, sample1] != -1))
# sample1_depths = np.rot90(np.repeat([population_size[:, sample1]], 2, axis=0))
sample1 = 50
sample2 = 120
print(sample1, sample2)
sample1_depths = population_size[:, sample1]
sample1_allele_counts = allele_counts[:, sample1, (allele_counts!=-1)[:, sample1].all(axis=0)]

# sample2_depths = np.rot90(np.repeat([population_size[:, sample2]], 2, axis=0))
sample2_depths = population_size[:, sample2]
sample2_allele_counts = allele_counts[:, sample2, (allele_counts!=-1)[:, sample2].all(axis=0)]

# print(sample1_depths[sample1_depths >= 5])
# print(sample1_allele_counts[(sample1_allele_counts / sample1_depths) > 0.01])
variants_to_include_1 = (sample1_depths > 5) #+ ((sample1_allele_counts / sample1_depths) > 0.01)
print(variants_to_include_1)
variants_to_include_2 = (sample2_depths > 5) #+ ((sample2_allele_counts / sample2_depths) > 0.01)
print(variants_to_include_2)
print(sample1_allele_counts[np.array([variants_to_include_1, variants_to_include_2]).all(axis=0)])
print(sample2_allele_counts[np.array([variants_to_include_1, variants_to_include_2]).all(axis=0)])

50 120
[ True  True  True ...  True  True  True]
[ True  True  True ... False False False]
[[ 0 34]
 [ 0 36]
 [ 0 34]
 ...
 [11 71]
 [16 80]
 [18 80]]
[[ 0  7]
 [ 0  8]
 [ 0  7]
 ...
 [ 0 15]
 [ 0 21]
 [ 0 21]]


In [117]:
total_fst = 0
for i in range(allele_counts.shape[0]):
    h1 = allel.HaplotypeArray([allele_counts[i, 150][allele_counts[i, 150] != -1]])
    a1 = allel.AlleleCountsArray([allele_counts[i, 150][allele_counts[i, 150] != -1]])
    #         print(h1)
    h2 = allel.HaplotypeArray([allele_counts[i, 60][allele_counts[i, 60] != -1]])
    a2 = allel.AlleleCountsArray([allele_counts[i, 60][allele_counts[i, 60] != -1]])
    #     print(h1)
    #     print(h2)
    num, den = allel.hudson_fst(h1, h2)
    fst = (num / den)
    #     print(fst)


    num, den = allel.hudson_fst(a1, a2)
    fst = (num / den)
#     print(fst)
    np.nan_to_num(fst, copy=False, nan=0.0, posinf=None, neginf=None)
    total_fst += fst[0]
print(total_fst / allele_counts.shape[0])

h1 = allel.HaplotypeArray([allele_counts[:, 150][allele_counts[:, 150] != -1]])
#         print(h1)
h2 = allel.HaplotypeArray([allele_counts[:, 60][allele_counts[:, 60] != -1]])
#     print(h1)
#     print(h2)
num, den = allel.hudson_fst(h1, h2)
fst = (num / den)

print(fst)
# hudson_fst_op(20, 30)

/tmp/ipykernel_17862/2743789114.py:11: RuntimeWarning: invalid value encountered in true_divide
  fst = (num / den)
/tmp/ipykernel_17862/2743789114.py:16: RuntimeWarning: invalid value encountered in true_divide
  fst = (num / den)


0.00104318834857757
[0.00201789]


In [112]:
a1 = allel.AlleleCountsArray([allele_counts[:, 150][allele_counts[:, 150] != -1]])
print(a1)
print(h1.count_alleles())
print(h1)

 1  0  0  0  0 ...  0  0  0  0  0

36429  5073  2361  1568  1314 ...     0     0     1     0     1

 1  0  0  0  0 ...  0  0  0  0  0



In [68]:
import os
os.makedirs('tests/', exist_ok=True)
os.symlink('../../../01-projects/', 'tests/projects', target_is_directory=True)

In [80]:
from Bio import SeqIO

dereplicated_clusters = pd.read_csv('/mnt/hpccs01/scratch/microbiome/n10853499/00-rosella_testing/00-CAMI_I/00-low_complexity/binning/test_cluster/data/dereplicated_clusters.txt', sep='\t', header=None)
bin_info = pd.read_csv('/mnt/hpccs01/scratch/microbiome/n10853499/00-rosella_testing/00-CAMI_I/00-low_complexity/binning/test_cluster/data/representative_checkm_full.tsv', sep='\t')

prev_cluster = ""
taxonomy = ""
sequences = []
for i in range(dereplicated_clusters.shape[0]):
    print(dereplicated_clusters.iloc[i, 1])
    if prev_cluster != dereplicated_clusters.iloc[i, 0]:
        if len(sequences) != 0:
            # print the fasta file
            representative_sample_name = prev_cluster.split("/")[-4]
            representative_name = prev_cluster.split("/")[-1].strip(".fna")
            representative_name = f"{representative_sample_name}/{representative_name}"
            print(representative_name)
            SeqIO.write(sequences, f"data/sequences_{representative_name}", "fasta")
        taxonomy = bin_info[bin_info['Bin Id'] == prev_cluster]['classification']
        prev_cluster = dereplicated_clusters.iloc[i, 0]
        sequences = []
    
    sample_name = prev_cluster.split("/")[-4]
    name = prev_cluster.split("/")[-1].strip(".fna")
    for record in SeqIO.parse(dereplicated_clusters.iloc[i, 1], "fasta"):
        record.id = f"{sample_name}_{name}_{record.id}"
        sequences.append(record)
        
        

/mnt/hpccs01/scratch/microbiome/n10853499/00-rosella_testing/00-CAMI_I/00-low_complexity/binning/test_dag/bins/final_bins/semibin_refined_bins.tsv.024.fna
test_dag_semibin_refined_bins.tsv.024_RL|S1|C1198
test_dag_semibin_refined_bins.tsv.024_RL|S1|C3912
test_dag_semibin_refined_bins.tsv.024_RL|S1|C10907
test_dag_semibin_refined_bins.tsv.024_RL|S1|C13173
test_dag_semibin_refined_bins.tsv.024_RL|S1|C13963
test_dag_semibin_refined_bins.tsv.024_RL|S1|C14639
test_dag_semibin_refined_bins.tsv.024_RL|S1|C15554
test_dag_semibin_refined_bins.tsv.024_RL|S1|C16754
/mnt/hpccs01/scratch/microbiome/n10853499/00-rosella_testing/00-CAMI_I/00-low_complexity/binning/test_dag/bins/final_bins/semibin_refined_bins.tsv.025.fna
test_dag_semibin_refined_bins.tsv.025_RL|S1|C1222
test_dag_semibin_refined_bins.tsv.025_RL|S1|C1432
test_dag_semibin_refined_bins.tsv.025_RL|S1|C2544
test_dag_semibin_refined_bins.tsv.025_RL|S1|C3352
test_dag_semibin_refined_bins.tsv.025_RL|S1|C3428
test_dag_semibin_refined_bins.tsv.

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/hpccs01/scratch/microbiome/n10853499/00-rosella_testing/00-CAMI_I/00-low_complexity/binning/test_dag/bins/final_bins/semibin_refined_bins.tsv.010_sub.fna'

In [135]:
a, b, c = allel.weir_cockerham_fst(test_vcf['calldata/GT'][:, :, 0], subpops=[[0, 1], [1, 2]])
print(a)
print(b)
print(c)
# a / (a + b + c)

ValueError: g must have three dimensions

In [189]:
allel.HaplotypeArray([[1, 1, 1, 1], [0, 0, 0, 0], [-1, -1, -1, -1]])

,0,1,2,3
0,1,1,1,1
1,0,0,0,0
2,.,.,.,.


In [147]:
subpops = [[0, 0], [1, 1], [2, 2]]
g = allel.HaplotypeArray(test_vcf['calldata/GT'][:, :, 0])
ac1 = g.count_alleles(subpop=subpops[0])
ac2 = g.count_alleles(subpop=subpops[1])
print(ac1)
print(ac2)
# print(ac1 == ac2)
allel.hudson_fst(ac1, ac2)[1]

0 2
0 2
0 2
...
0 2
0 2
0 2

0 2
0 2
0 2
...
0 2
0 2
0 2



array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       nan, nan,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,
        1.,  1.,  1.,  0., nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [127]:
fst = np.sum(a) / (np.sum(a) + np.sum(b) + np.sum(c))
fst.sum()

nan

In [58]:
g = np.array([[[0, 0], [0, 0], [1, 1], [1, 1]],
     [[0, 1], [0, 1], [0, 1], [0, 1]],
     [[0, 0], [0, 0], [0, 0], [0, 0]],
     [[0, 1], [1, 2], [1, 1], [2, 2]],
     [[0, 0], [1, 1], [0, 1], [-1, -1]]])
a, b, c = allel.weir_cockerham_fst(g, subpops=[[0, 1], [2, 3]])
a / (a + b + c)

/tmp/ipykernel_95597/879179919.py:7: RuntimeWarning: invalid value encountered in true_divide
  a / (a + b + c)


array([[ 1. ,  1. ,  nan],
       [ 0. ,  0. ,  nan],
       [ nan,  nan,  nan],
       [ 0. , -0.5, -0.5],
       [-1.8, -1.8,  nan]])

In [26]:
batch = pd.read_csv("example_batch.tsv", sep=None)
# batch = pd.read_csv("batch2.txt", sep=None)

for i in range(batch.shape[0]):
    print(f"Processing {batch.iloc[i, 0]}")
    
    sample = batch.iloc[i, 0]
    if isinstance(sample, str):
        sample = sample.strip()
    else:
        sample = None
    
    s1 = batch.iloc[i, 1]
    if isinstance(s1, str):
        s1 = s1.strip().split(',')
    else:
        s1 = None
        
    s2 = batch.iloc[i, 2]
    if isinstance(s2, str):
        s2 = s2.strip().split(',')
    else:
        s2 = None
    
    l = batch.iloc[i, 3]
    if isinstance(l, str):
        l = l.strip().split(',')
    else:
        l = None
    
    l_type = batch.iloc[i, 4].strip()
    assembly = batch.iloc[i, 5]
    
    if isinstance(assembly, str):
        assembly = assembly.strip()
    else:
        assembly = None
    
    print(f"{sample} {s1} {s2} {l} {assembly}")

Processing sample_1
sample_1 ['pe1.fq.gz'] ['pe2.fq.gz'] ['nanopore.fq.gz'] None
Processing sample_2
sample_2 ['interleaved.fq.gz'] None ['pacbio.fq.gz'] None
Processing sample_3
sample_3 ['pe1.1.fq.gz', 'pe1.2.fq.gz'] ['pe2.1.fq.gz', 'pe2.2.fq.gz'] ['n1.fq.gz', 'n2.fq.gz', 'n3.fq.gz'] None
Processing sample_4
sample_4 ['pe1.1.fq.gz', 'pe1.2.fq.gz'] ['pe2.1.fq.gz', 'pe2.2.fq.gz'] ['n1.fq.gz', 'n2.fq.gz', 'n3.fq.gz'] assembly.fasta


/tmp/ipykernel_17862/4291521788.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  batch = pd.read_csv("example_batch.tsv", sep=None)


In [36]:
# any(i in [1, 2, 3] for i in [3, 4, 5, 6])
test = [2, 3, 4, 5]
inds = [idx for idx, i in enumerate([1, 3, 4, 5]) if i in [2, 3]]
test.insert(inds[0] + 1, 0)
print(test)

[2, 3, 0, 4, 5]


In [46]:
reads = "data/short_reads.racon.1.fastq.gz ".strip().split(",")
print(reads)
reads[-3::] == ".gz"

['data/short_reads.racon.1.fastq.gz']


False